## Spin Up added! Reflective Boundary Condition.

2025.03.28.

#### problem setup

The purpose of this script is to step-by-step implement the 3 PDEs describing mosquito's behaviour in FEniCSx. For more details, please refer to our pre-print.

stagnated water field and wind field shall all be supplied as NETCDF files (.nc).


### (1) The simplified testcase with three equations


$\frac{\partial M}{\partial t} - \nabla \cdot (D \nabla M) + \mu_1 M - \gamma A + c_{v} \nabla \cdot (\vec{v} M)= 0$

$\frac{\partial E}{\partial t} - rM + eE= 0$

$\frac{\partial A}{\partial t} - e(1-\frac{A}{k})E + (\mu_2 + \gamma ) A= 0$

where M is the mobile phase population density, E is the egg phase population density, A is the aquatic phase population density, D is the diffusion coefficient ($m^2$/day), $\mu_1$ is the mobile phase mortility rate (/day), $\mu_2$ is the immobile phase mortility rate (/day), r is the oviation rate of females, e is the hatching rate, $\gamma$ is the immobile phase maturation rate, k is the carrying capacity (1/$m^2$), $c_v$ is the coefficient of convection caused by wind.




### (2) Variational Formulation

this section will be implemented following: https://fenicsproject.org/pub/tutorial/html/._ftut1010.html

Numerical Scheme: backward Euler and Finite Element.

Define the domain: $\Omega = [0,1] \times [0,1]$

Boundary condition: Dirichlet, $M|_{\partial \Omega} = 0$, $E|_{\partial \Omega} = 0$, $A|_{\partial \Omega} = 0$.


(Comparing to Forward Euler, we choose backward Euler since we will start with a large time step.)




With backward Euler, denote $M^n = M(x,y,t^n)$

$ \frac{M^{n+1}-M^n}{\Delta t}- \nabla \cdot (D \nabla M^{n+1}) + \mu_1 M^{n+1} -\gamma A^{n+1} + c_{v} \nabla \cdot (\vec{v} M^{n+1}) = 0$, $\Omega \times (0,T]$


$ \frac{E^{n+1}-E^n}{\Delta t}- rM^{n+1} + eE^{n+1} = 0$, $\Omega \times (0,T]$


$ \frac{A^{n+1}-A^n}{\Delta t}- e(1-\frac{A^{n+1}}{k})E^{n+1} + (\mu_2 + \gamma ) A^{n+1}= 0$, $\Omega \times (0,T]$


Total time: T = 1 day, steps = 50, and we use timestep $\Delta t = T/steps$.

#### > Variational Formulation

$V=\{v\in H^1(\Omega)| v=0 $ on $ \partial \Omega\}$

$\hat{V}=\{v\in H^1(\Omega)| v=0 $ on $ \partial \Omega\}$

Find $u_1^{n+1}, u_2^{n+1}, u_3^{n+1} \in V$ such that 

$\int_\Omega (\frac{u_1^{n+1}-u_1^n}{\Delta t}v_1 +D\nabla u_1^{n+1} \cdot \nabla v_1 + \mu_1 u_1^{n+1} v_1 + \gamma u_3^{n+1} v_1 + c_{v} \nabla \cdot (\vec{v} u_1^{n+1})v_1)\textbf{dx} = 0$,    

$\int_\Omega (\frac{u_2^{n+1}-u_2^n}{\Delta t}v_2 -r u_1^{n+1}v_2 + e u_2^{n+1}v_2)\textbf{dx} = 0$,     

$\int_\Omega (\frac{u_3^{n+1}-u_3^n}{\Delta t}v_3 -e(1-\frac{u_3^{n+1}}{k})u_2^{n+1}v_3 + (\mu_2 + \gamma ) u_3^{n+1}v_3)\textbf{dx} = 0$,      

$\forall v_1, v_2, v_3 \in \hat{V}$



Organize them into one large equation:

$\int_\Omega (\frac{u_1^{n+1}-u_1^n}{\Delta t}v_1 +D\nabla u_1^{n+1} \cdot \nabla v_1 + \mu_1 u_1^{n+1} v_1 + \gamma u_3^{n+1} v_1 + c_{v} \nabla \cdot (\vec{v} u_1^{n+1})v_1))\textbf{dx} + \int_\Omega (\frac{u_2^{n+1}-u_2^n}{\Delta t}v_2 -r u_1^{n+1}v_2 + e u_2^{n+1}v_2)\textbf{dx} + \int_\Omega (\frac{u_3^{n+1}-u_3^n}{\Delta t}v_3 -e(1-\frac{u_3^{n+1}}{k})u_2^{n+1}v_3 + (\mu_2 + \gamma ) u_3^{n+1}v_3)\textbf{dx} = 0$


#### > Apply finite element method by instead finding the u, v in discretize space $V_h \subset V, \hat{V_h} \subset \hat{V}$. 

In FEniCSX, we choose first order Lagrange as function space.

Find $u_{1h}^{n+1}, u_{2h}^{n+1},u_{3h}^{n+1}\in V_h$ such that 

$\int_\Omega (\frac{u_{1h}^{n+1}-u_{1h}^n}{\Delta t}v_1 +D\nabla u_{1h}^{n+1} \cdot \nabla v_{1h} + \mu_1 u_{1h}^{n+1} v_1 + \gamma u_{3h}^{n+1} v_1 )\textbf{dx} + \int_\Omega (\frac{u_{2h}^{n+1}-u_{2h}^n}{\Delta t}v_2 -r u_{1h}^{n+1}v_2 + e u_{2h}^{n+1}v_2)\textbf{dx} + \int_\Omega (\frac{u_{3h}^{n+1}-u_{3h}^n}{\Delta t}v_3 -e(1-\frac{u_{3h}^{n+1}}{k})u_{2h}^{n+1}v_3 + (\mu_2 + \gamma ) u_{3h}^{n+1}v_3)\textbf{dx} = 0$, $\forall v_{1h}, v_{2h}, v_{3h} \in \hat{V_h}$





### Reflective Boundary Condition is applied for Spin-up.

[from perplexity]

Advantages:
* Realistic Representation of Physical Barriers: Reflective conditions can effectively model physical barriers or boundaries in a geographical area, such as coastlines or mountain ranges, where mosquitoes might "bounce off" or change direction upon encountering these barriers.
* Applicability to Heterogeneous Landscapes: Reflective conditions can better capture the dynamics of mosquito populations in areas with diverse habitats or barriers, as they allow for the simulation of realistic boundary interactions.

Disadvantages:
* Potential for Unrealistic Behavior: If not properly implemented, reflective conditions might lead to unrealistic behavior, such as mosquitoes accumulating at boundaries without dispersing further.
* Increased Complexity: Reflective conditions can introduce additional complexity in terms of modeling the interactions between mosquitoes and the environment at these boundaries.

## Gaussian Random Field Range for Spin Up

u_1: mobile phase, u_2: egg phase, u_3: aquatic phase

{'u_1_min_val': 0.0, 'u_1_max_val': 0.02, 
                    'u_2_min_val': 0.0, 'u_2_max_val': 0.02, 
                    'u_3_min_val': 0.0, 'u_3_max_val': 0.02}

In [1]:
import sys
import os
# Get the path of the current directory (examples)
current_dir = os.path.dirname(os.path.abspath('mosquito_problem_spinup_wind.ipynb'))
# Construct the path to the src directory
src_dir = os.path.join(current_dir, '..', 'src')
# Add the src directory to the Python path
sys.path.insert(0, src_dir)

import mosquito_solver as ms
import time

'period': the total number of days (float) to run the simulation;
'steps': total number of time steps;
'nx': total number of cells in x-direction in the mesh;
'ny': total number of cells in y-direction in the mesh;

'stations': the np.array of np.array of coordinates (latitude, longitude) of all the traps locations;
'station_observe_days': the np.array of [t_start, t_end], which tells MOFES between which two days (float) shall it perform integration and compute the simulated trapped mosquito numbers. 

All the other parameters are the model parameters as stated in the manuscript.

In [ ]:
spin_up_problem_setting = ms.Neches_problem_settings(diffusion_coefficient=18969, # m^2/day
                                                  mu_1 = 0.1177,  # 1/day 
                                                  mu_2 =  0.0250,  # 1/day
                                                  oviation_rate = 34,  # 1/day
                                                  hatching_rate = 0.24, # 1/day
                                                  immobile_maturation_rate = 0.0625,  # 1/day 
                                                  carrying_capacity = 0.0590, # 1/m^2
                                                  constant_for_mobile = 0.01,                                                   
                                                  period = 4.0, 
                                                  steps = 100, 
                                                  nx = 250, 
                                                  ny = 250, 
                                                  save_path = '/Your_path/MOFES/results/spin_up_problem_setting6_4daySpinUp/', 
                                                  output_name = '250_250_10_wind0.004', 
                                                  stag_path = '/Your_path/stag-water/', 
                                                  flag_spinup = True,
                                                  flag_advection=True,
                                                  flag_observation=True,
                                                  wind_path='/Your_path/wind_Harvey/14d495d5fc8e218b01eca6895328d3ab.nc',
                                                  constant_for_wind = 0.004, 
                                                  stations=[[30.09694,-94.1052],[30.0617,-94.12863],[30.06157,-94.13016],[30.03911,-94.0026],[30.06717,-94.12863],[30.12529,-94.11944],[29.96163,-94.04143],[30.0558,-94.09067],[30.03911,-94.0926]], 
                                                  station_observe_days=[2.0,4.0])

spin_up_solver_setting = ms.Cartesian_MosquitoSolver(spin_up_problem_setting)
start = time.time()
spin_up_solver_setting.spin_up()
spin_up_solver_setting.solve_mosquito()
end = time.time()
print('Time elapsed: ', end - start)

Coordinates outside the domain:
(30.0617, -94.12863)
(30.06157, -94.13016)
(30.06717, -94.12863)
Station coordinates within the domain: (30.09694, -94.1052)
Station coordinates within the domain: (30.03911, -94.0026)
Station coordinates within the domain: (30.12529, -94.11944)
Station coordinates within the domain: (29.96163, -94.04143)
Station coordinates within the domain: (30.0558, -94.09067)
Station coordinates within the domain: (30.03911, -94.0926)
Directory '/Users/liting/Documents/GitHub/fenics-mos/dirichlet_results/spin_up_problem_setting6_4daySpinUp//video/' exists.
Save mesh points.
Spin up started.
start to generate random field...
start to build coarse grid...
advised coarse grid size: 62 67
applied grid size: 62 67
start to build covariance matrix...
start to interpolate to fine mesh...
start to generate random field...
start to build coarse grid...
advised coarse grid size: 62 67
applied grid size: 62 67
start to build covariance matrix...
start to interpolate to fine me